### **Knn-Imputation**

It is multivarient impuation. We use all the columns avilable in the dataset  to fill the missing values in the knn impuation. 
for eg in the dataset:
|**X1**|**X2**|**X3**|
|-------|---|-----|
|1|2|3|
|2|nan|5|
|5|nan|nan|
|45|44|33|
|33|32|nan|
|nan|32|43|
|21|33|27|

Here for the impuation, we fill the missing values with the value of the row have the lowest distance among the nearest neighbours. The value of the k depends upon the number of the closet distance we take. For the distance caluclation, non euclidain distance is used. \
$ non-euclidan-distance=\sqrt{weight*((x_1-x_2)^2+(y_1-y_2)^2+...)}$

Here the weight depends upon the missing values:\
$weight=\frac{num-of -total -observations- in -a -row}{num -of -non -missing -observations}$


distance of the second row to the first can be calculated as :\
$ dis=\sqrt{\frac{3}{2}((2-1)^2+(5-3)^2)}$

<u>Advanatages</u>
* More accuarate\
<u>Disadvantages</u>
* Takes more time
* Provide the whole training set into the production.

In [182]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split

In [183]:
titanic=sns.load_dataset('titanic')[['age','fare','pclass','survived']]

In [184]:
titanic.head()

,age,fare,pclass,survived
0,22.0,7.2500,3,0
1,38.0,71.2833,1,1
2,26.0,7.9250,3,1
3,35.0,53.1000,1,1
4,35.0,8.0500,3,0


In [185]:
titanic.isnull().sum()

age         177
fare          0
pclass        0
survived      0
dtype: int64

In [186]:
X=titanic.drop(columns=['survived'])
Y=titanic['survived']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=2)

In [187]:
knn=KNNImputer(n_neighbors=2,weights='distance')
x_train_trf=knn.fit_transform(X_train)
x_test_trf=knn.transform(X_test)

In [188]:
pd.DataFrame(x_train_trf,columns=X_train.columns)

,age,fare,pclass
0,40.000000,27.7208,1.0
1,4.000000,16.7000,3.0
2,47.000000,9.0000,3.0
3,9.000000,31.3875,3.0
4,20.000000,9.8458,3.0
...,...,...,...
707,30.000000,8.6625,3.0
708,30.631313,8.7125,3.0
709,71.000000,49.5042,1.0
710,28.000000,221.7792,1.0


In [189]:
lr=LogisticRegression()
lr.fit(x_train_trf,Y_train)
y_pred=lr.predict(x_test_trf)

In [190]:
accuracy_score(y_pred,Y_test)

0.7094972067039106

In [191]:
from sklearn.impute import SimpleImputer
si=SimpleImputer()
x_train_trf2=si.fit_transform(X_train)
x_test_trf2=si.fit_transform(X_test)

In [192]:
lr=LogisticRegression()
lr.fit(x_train_trf2,Y_train)
y_pred2=lr.predict(x_test_trf2)
accuracy_score(y_pred2,Y_test)

0.6927374301675978

In [193]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [194]:
pipe=Pipeline(steps=[
    ('imputer1',KNNImputer(n_neighbors=4,weights='distance')),
    ('lf',LogisticRegression())
])
cross_val_score(pipe,X,Y,cv=4,scoring='accuracy').mean()

np.float64(0.7014856785036157)

Here the weights='distance' will select the file based on distance as the weight during the point calculation.\
for eg: lets we have two nearest distacne be 10,30 then the point will be calculated as:\
${\frac{1}{10}*x_1}+{\frac{1}{30}*x_2}$, for the value of k 2\
for the weights='uniform' we take the mean  of the points with the nearest distance as per the number of k.

In [195]:
pipe2=Pipeline(steps=[
    ('imputer2',SimpleImputer()),
    ('lf2',LogisticRegression())
])
print(cross_val_score(pipe2,X,Y,cv=4,scoring='accuracy').mean())

0.6992485759301903


We see the model performance is better with the simple imputer.